1. Здесь идет установка **необходимых библиотек**

In [1]:
!pip3 install pydub
!pip3 install vosk
!pip3 install torch
!pip3 install transformers
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=35d48f9af49639cd4a252e42c8f1cd45bd974f94256fd05f514567d2a1e800de
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt
Cloning into 'ffmpeg'...
remote: Enumerating objects: 19343, done.
remote: Counting objects: 100% (19343/19343), done.
remote: Compressing objects: 100% (11622/11622), done.
remote: Total 707407 (delta 14284), reused 10093 (delta 7662)
Receiving objects: 100% (707407/707407), 175.22 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (571475/571475), done.
Updating files: 100% (8280/8280), done.


1.2 Vosk model installation

In [2]:
!curl -o ./model.zip https://alphacephei.com/vosk/models/vosk-model-ru-0.42.zip
!unzip model.zip
!mv vosk-model-ru-0.42/ model
!rm -rf model.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1847M  100 1847M    0     0  24.3M      0  0:01:15  0:01:15 --:--:-- 24.8M
Archive:  model.zip
   creating: vosk-model-ru-0.42/
   creating: vosk-model-ru-0.42/graph/
  inflating: vosk-model-ru-0.42/graph/words.txt  
   creating: vosk-model-ru-0.42/graph/phones/
 extracting: vosk-model-ru-0.42/graph/phones/silence.csl  
  inflating: vosk-model-ru-0.42/graph/phones/align_lexicon.txt  
  inflating: vosk-model-ru-0.42/graph/phones/word_boundary.txt  
 extracting: vosk-model-ru-0.42/graph/phones/optional_silence.csl  
 extracting: vosk-model-ru-0.42/graph/phones/optional_silence.txt  
  inflating: vosk-model-ru-0.42/graph/phones/disambig.int  
 extracting: vosk-model-ru-0.42/graph/phones/disambig.txt  
  inflating: vosk-model-ru-0.42/graph/phones/align_lexicon.int  
  inflating: vosk-model-ru-0.42/graph/phones/word_boundary.int  


This model can be put to VOSK, but its use takes too much resources of the computer itself. Deals with punctuation marks

In [5]:
!curl -o recasepunc.zip https://alphacephei.com/vosk/models/vosk-recasepunc-ru-0.22.zip
!unzip recasepunc.zip
!mv vosk-recasepunc-ru-0.22/ recasepunc
!rm -rf recasepunc.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  1 1622M    1 23.9M    0     0  10.1M      0  0:02:39  0:00:02  0:02:37 10.1M^C
Archive:  recasepunc.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of recasepunc.zip or
        recasepunc.zip.zip, and cannot find recasepunc.zip.ZIP, period.
mv: cannot stat 'vosk-recasepunc-ru-0.22/': No such file or directory


In [7]:
#import libraries
from vosk import Model, KaldiRecognizer, SetLogLevel
from pydub import AudioSegment
import subprocess
import json
import os
from moviepy.editor import VideoFileClip
from moviepy.video.io.html_tools import ffmpeg_parse_infos


In [7]:
#def converttomp3(mp4file, wavfile):
    video=VideoFileClip(mp4file)
    # получаем аудиодорожку
    audio=video.audio
    # сохраняем аудио файл
    audio.write_audiofile(wavfile)
    # уничтожаем объекты
    # что бы не было ошибок
    audio.close()
    video.close()

#converttomp3('video.mp4', 'file.wav')


MoviePy - Writing audio in file.wav


MoviePy - Done.


In [8]:
from moviepy.editor import VideoFileClip
def convert_mp4_to_wav(input_file, output_file):
    video = VideoFileClip(input_file)
    audio = video.audio
    audio.write_audiofile(output_file)

input_file = 'video.mp4'
output_file = 'file.wav'
convert_mp4_to_wav(input_file, output_file)


MoviePy - Writing audio in file.wav


MoviePy - Done.


In [9]:
#SetLogLevel(0)
import os
# checking model
if not os.path.exists("model"):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")
    exit (1)

# set Frame Rate
FRAME_RATE = 16000
CHANNELS=1

model = Model("model")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

# Используя библиотеку pydub делаем предобработку аудио
mp3 = AudioSegment.from_mp3('file.wav')
mp3 = mp3.set_channels(CHANNELS)
mp3 = mp3.set_frame_rate(FRAME_RATE)


In [ ]:
# Преобразуем вывод в json
rec.AcceptWaveform(mp3.raw_data)
result = rec.Result()
text = json.loads(result)["text"]

# Добавляем пунктуацию (Занимает очень много ресурсов, можно и без нее)
#cased = subprocess.check_output('python3 recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)


In [ ]:
# Записываем результат в файл "data.txt"
with open('ff.txt', 'w') as f:
    json.dump(text, f, ensure_ascii=False, indent=4)